# Phase 3: Detailed Planning for Each Request Checkpoint

This notebook handles:
1. Loading results from Phase 1 and Phase 2
2. For each checkpoint, generating detailed execution specifications
3. Creating task tranches with low-level implementation tasks
4. Defining file contexts (beginning and ending states)
5. Saving detailed checkpoints for Phase 4

In [1]:
# Import shared utilities
import sys
sys.path.append('.')

In [2]:
from google.genai import types
from mirascope.core import google
from mirascope import Messages
from pydantic import BaseModel, Field
from typing import List, Optional, Literal
import os

from utils.interim_data_management import load_interim_data, save_interim_data
from utils.config import get_config

## Load Previous Phase Results

In [3]:
# Get config
config = get_config()

# Load Phase 1 data
raw_inputs = load_interim_data(config.get("INTERIM_DATA_DIR"), "raw_inputs", "phase1")
files_dict = load_interim_data(config.get("INTERIM_DATA_DIR"), "files_dict", "phase1")
file_analyses = load_interim_data(config.get("INTERIM_DATA_DIR"), "file_analyses", "phase1")

# Load Phase 2 data
checkpoint_outline_data = load_interim_data(config.get("INTERIM_DATA_DIR"), "checkpoint_outline", "phase2")

# Extract key variables
INPUT_NAME = raw_inputs["input_name"]
user_instructions = raw_inputs["user_instructions"]
file_tree = raw_inputs["file_tree"]
GLOBAL_CODEBASE_PURPOSE = raw_inputs["global_codebase_purpose"]

print(f"Loaded data from previous phases:")
print(f"- Files: {len(files_dict)}")
print(f"- File analyses: {len(file_analyses)}")
print(f"- Checkpoints to process: {len(checkpoint_outline_data['request_checkpoints'])}")

Loaded raw_inputs from interim_data/phase1/raw_inputs.json
Loaded files_dict from interim_data/phase1/files_dict.json
Loaded file_analyses from interim_data/phase1/file_analyses.json
Loaded checkpoint_outline from interim_data/phase2/checkpoint_outline.json
Loaded data from previous phases:
- Files: 55
- File analyses: 55
- Checkpoints to process: 7


In [4]:
# Reconstruct Pydantic models
class InitialFileAnalysisItemWithContent(BaseModel):
    file_path: str
    overall_relevance_to_request_objective: str
    initial_content_summary_purpose: str
    file_contents: str

class RequestCheckpointOutline(BaseModel):
    order: int
    title: str
    goal_for_checkpoint: str
    prerequisites: Optional[List[str]] = Field(default_factory=list)
    expected_outcome: Optional[str] = None

class FullRequestCheckpointOutline(BaseModel):
    request_checkpoints: List[RequestCheckpointOutline]

# Convert loaded data back to models
all_analyses_with_content = [
    InitialFileAnalysisItemWithContent(**item) for item in file_analyses
]

request_checkpoint_outline = FullRequestCheckpointOutline(**checkpoint_outline_data)

## Define Detailed Checkpoint Models

In [5]:
class LowLevelTask(BaseModel):
    """Represents a single, atomic, low-level action to be performed."""
    task_id: str = Field(
        description="A unique identifier for the task (e.g., '1.1.1')"
    )
    file_path: str = Field(
        description="The path to the file that this task will modify."
    )
    description: str = Field(
        description="A human-readable description of what this task aims to achieve."
    )
    action_details: str = Field(
        description="The specific command or instruction to be executed."
    )
    complexity: int = Field(
        description="A number between 1 (trivial) and 10 (very complex) indicating the complexity."
    )
    depends_on: Optional[List[str]] = Field(
        default_factory=list,
        description="A list of 'task_id's that must be completed before this task."
    )
  
class TaskTranche(BaseModel):
    """A collection of LowLevelTasks grouped together."""
    tranche_id: str = Field(
        description="A unique identifier for the tranche (e.g., '1.1')."
    )
    goal: str = Field(
        description="The specific sub-goal this tranche of tasks aims to achieve."
    )
    low_level_tasks: List[LowLevelTask] = Field(
        description="An ordered list of low-level tasks to be executed."
    )

class RequestCheckpointExecutionSpecification(BaseModel):
    """The detailed plan for how to achieve a specific Request Checkpoint."""
    implementation_notes_specific_to_checkpoint: Optional[List[str]] = Field(
        default_factory=list,
        description="Technical details, constraints, or guidance specific to this checkpoint."
    )
    task_tranches: List[TaskTranche] = Field(
        description="An ordered list of task tranches required to fulfill the checkpoint's objective."
    )

class RequestCheckpointFileContextItem(BaseModel):
    """Represents a file's state within the context of a Request Checkpoint."""
    file_path: str = Field(
        description="The absolute or relative path to the file."
    )
    status_description: str = Field(
        description="A description of the file's status."
    )

class RequestCheckpointSpecificFileContext(BaseModel):
    """Manages files relevant at the beginning and expected at the end of a checkpoint."""
    beginning_files_for_request_checkpoint: List[RequestCheckpointFileContextItem] = Field(
        description="Files and their statuses at the start of this checkpoint."
    )
    ending_files_for_request_checkpoint: List[RequestCheckpointFileContextItem] = Field(
        description="Files and their expected statuses after this checkpoint."
    )

class RequestCheckpoint(BaseModel):
    """A major, ordered milestone in the process of fulfilling the overall request."""
    order: int
    title: str
    goal_for_request_checkpoint: str
    prerequisites: Optional[List[str]] = Field(default_factory=list)
    expected_outcome_or_deliverables: Optional[str] = None
    request_checkpoint_specific_file_context: RequestCheckpointSpecificFileContext
    execution_spec: RequestCheckpointExecutionSpecification

## Load Spec System Prompt

In [6]:
def load_spec_system_prompt() -> str:
    """Loads the spec system prompt from the file system."""
    prompt_path = os.path.join(config.get("PROMPTS_DIR"), "spec_system.md")
    with open(prompt_path, 'r') as f:
        spec_system_content = f.read()
    return spec_system_content

# Load the spec system prompt
SPEC_SYSTEM_PROMPT = load_spec_system_prompt()
print(f"Loaded spec system prompt: {len(SPEC_SYSTEM_PROMPT)} characters")

Loaded spec system prompt: 67352 characters


## Define Checkpoint Builder Prompt

In [7]:
CHECKPOINT_BUILDER_USER_PROMPT = """<user_prompt>
    <input_text>
        - As a Spec Prompt Engineer and Checkpoint Execution Strategist, please analyze the provided inputs and generate a comprehensive specification document.
        - Ensure you follow and complete all phases of the `<internal_workflow>` to produce the full specification.
        - Take all the time needed to generate the best possible specification document; there is no rush.
    </input_text>
    <app_purpose_input>
        {app_purpose_content_placeholder}
    </app_purpose_input>
    <high_level_objective>
        {user_requests_content_placeholder}
    </user_requests_input>
    <file_tree_input>
        {file_tree_content_placeholder}
    </file_tree_input>
    <all_file_analysis>
        {relevant_files_content_placeholder}
    </all_file_analysis>
    <previous_request_checkpoint_optional>
        {previous_request_checkpoint_optional_placeholder}
    </previous_request_checkpoint_optional>
    <current_request_checkpoint_shell>
        {request_checkpoint_shell}
    </current_request_checkpoint_shell>
    <final_thoughts>Take a deep breath, and please begin constructing the specification prompt based on these inputs.</final_thoughts>
</user_prompt>
"""

def format_checkpoint_spec_user_prompt(
    app_purpose_content: str,
    user_requests_content: str,
    file_tree_content: str,
    relevant_files_content: str,
    request_checkpoint_shell: str,
    previous_request_checkpoint_optional: str = None
) -> str:
    """Formats the user prompt for the checkpoint builder."""
    user_prompt = CHECKPOINT_BUILDER_USER_PROMPT.format(
        app_purpose_content_placeholder=app_purpose_content.strip(),
        user_requests_content_placeholder=user_requests_content.strip(),
        file_tree_content_placeholder=file_tree_content.strip(),
        relevant_files_content_placeholder=relevant_files_content.strip(),
        request_checkpoint_shell=request_checkpoint_shell.strip(),
        previous_request_checkpoint_optional_placeholder=previous_request_checkpoint_optional
    )
    return user_prompt

## Generate Detailed Checkpoints

In [8]:
# Configure model
model_name = "gemini-2.5-pro-preview-05-06"
@google.call(
    model_name,
    response_model=RequestCheckpoint,
    call_params={
        "config": types.GenerateContentConfig(
            thinking_config=types.ThinkingConfig(
                thinking_budget=config.get("THINKING_BUDGET")
            ),
            temperature=config.get("TEMPERATURE")
        )
    },
)
def generate_checkpoint(system_prompt: str, user_prompt: str) -> str:
    return [Messages.System(system_prompt), Messages.User(user_prompt)]

In [9]:
# Process each checkpoint
previous_request_checkpoint = None
generated_checkpoints = []

# Allow limiting the number of checkpoints to process (for testing)
MAX_CHECKPOINTS_TO_PROCESS = None  # Set to a number to limit, or None to process all
START_FROM_CHECKPOINT = None  # Set to a number to start from that checkpoint (1-based), or None to start from beginning

checkpoints_to_process = request_checkpoint_outline.request_checkpoints
if MAX_CHECKPOINTS_TO_PROCESS:
    checkpoints_to_process = checkpoints_to_process[:MAX_CHECKPOINTS_TO_PROCESS]

# Load previous checkpoint if starting from middle
if START_FROM_CHECKPOINT and START_FROM_CHECKPOINT > 1:
    checkpoint_output_dir = os.path.join(config.get("INTERIM_DATA_DIR"), "generated_checkpoints", INPUT_NAME)
    prev_checkpoint_path = os.path.join(checkpoint_output_dir, f"checkpoint_{START_FROM_CHECKPOINT-1:02d}.json")
    if os.path.exists(prev_checkpoint_path):
        with open(prev_checkpoint_path, "r") as f:
            previous_request_checkpoint = f.read()
        # Load existing checkpoints
        generated_checkpoints = []
        for i in range(1, START_FROM_CHECKPOINT):
            checkpoint_path = os.path.join(checkpoint_output_dir, f"checkpoint_{i:02d}.json")
            with open(checkpoint_path, "r") as f:
                generated_checkpoints.append(RequestCheckpoint.model_validate_json(f.read()))
        print(f"Loaded {len(generated_checkpoints)} previous checkpoints")
        # Slice remaining checkpoints
        checkpoints_to_process = checkpoints_to_process[START_FROM_CHECKPOINT-1:]
    else:
        print(f"Warning: Could not find previous checkpoint at {prev_checkpoint_path}")
        print("Starting from beginning...")
        START_FROM_CHECKPOINT = None

print(f"Processing {len(checkpoints_to_process)} checkpoints...\n")

for i, checkpoint_outline in enumerate(checkpoints_to_process, start=START_FROM_CHECKPOINT or 1):
    print(f"Processing checkpoint {i}/{len(request_checkpoint_outline.request_checkpoints)}: {checkpoint_outline.title}")
    
    # Format the user prompt
    checkpoint_outline_user_prompt = format_checkpoint_spec_user_prompt(
        GLOBAL_CODEBASE_PURPOSE, 
        user_instructions, 
        file_tree, 
        '\n'.join([analysis.model_dump_json(indent=2) for analysis in all_analyses_with_content]),
        checkpoint_outline.model_dump_json(indent=2),
        previous_request_checkpoint
    )
    
    # Generate the detailed checkpoint
    generated_checkpoint = generate_checkpoint(SPEC_SYSTEM_PROMPT, checkpoint_outline_user_prompt)
    generated_checkpoints.append(generated_checkpoint)
    
    # Save individual checkpoint
    checkpoint_output_dir = os.path.join(config.get("INTERIM_DATA_DIR"), "generated_checkpoints", INPUT_NAME)
    os.makedirs(checkpoint_output_dir, exist_ok=True)
    
    checkpoint_filename = f"checkpoint_{i:02d}.json"
    checkpoint_path = os.path.join(checkpoint_output_dir, checkpoint_filename)
    with open(checkpoint_path, "w") as f:
        f.write(generated_checkpoint.model_dump_json(indent=2))
    
    print(f"  ✓ Saved to: {checkpoint_path}")
    
    # Update previous checkpoint for next iteration
    previous_request_checkpoint = generated_checkpoint.model_dump_json(indent=2)
    
    # Optional: Add a break here if you want to process just one checkpoint for testing
    # break

print(f"\nGenerated {len(generated_checkpoints)} detailed checkpoints")

Processing 7 checkpoints...

Processing checkpoint 1/7: Implement New Server Actions for Context Bundles


/Users/Ford/anaconda3/envs/promo/lib/python3.12/site-packages/google/genai/_common.py:240: UserWarning: null is not a valid Type
  warnings.warn(f"{value} is not a valid {cls.__name__}")


  ✓ Saved to: interim_data/generated_checkpoints/context_manager_update_frontend/checkpoint_01.json
Processing checkpoint 2/7: Refactor Legacy `resolveContextSelectionAction`
  ✓ Saved to: interim_data/generated_checkpoints/context_manager_update_frontend/checkpoint_02.json
Processing checkpoint 3/7: Implement Frontend State Management for Active Context Bundle
  ✓ Saved to: interim_data/generated_checkpoints/context_manager_update_frontend/checkpoint_03.json
Processing checkpoint 4/7: Integrate Context Bundle ID into Frontend Operations and State
  ✓ Saved to: interim_data/generated_checkpoints/context_manager_update_frontend/checkpoint_04.json
Processing checkpoint 5/7: Implement Frontend Display of Context Bundle Snapshots on Cards
  ✓ Saved to: interim_data/generated_checkpoints/context_manager_update_frontend/checkpoint_05.json
Processing checkpoint 6/7: Update Documentation to Reflect Server-Action Architecture
  ✓ Saved to: interim_data/generated_checkpoints/context_manager_upda

## Save Phase 3 Results

In [10]:
# Save all generated checkpoints
save_interim_data(config.get("INTERIM_DATA_DIR"), generated_checkpoints, "generated_checkpoints", "phase3")

# Create a summary of the generated checkpoints
phase3_summary = {
    "checkpoints_generated": len(generated_checkpoints),
    "checkpoint_details": []
}

for checkpoint in generated_checkpoints:
    checkpoint_detail = {
        "order": checkpoint.order,
        "title": checkpoint.title,
        "tranches_count": len(checkpoint.execution_spec.task_tranches),
        "total_tasks": sum(len(tranche.low_level_tasks) for tranche in checkpoint.execution_spec.task_tranches),
        "beginning_files": len(checkpoint.request_checkpoint_specific_file_context.beginning_files_for_request_checkpoint),
        "ending_files": len(checkpoint.request_checkpoint_specific_file_context.ending_files_for_request_checkpoint)
    }
    phase3_summary["checkpoint_details"].append(checkpoint_detail)

save_interim_data(config.get("INTERIM_DATA_DIR"), phase3_summary, "phase3_summary", "phase3")

# Display summary
print("\nPhase 3 Summary:")
print("=" * 50)
for detail in phase3_summary["checkpoint_details"]:
    print(f"\nCheckpoint {detail['order']}: {detail['title']}")
    print(f"  - Tranches: {detail['tranches_count']}")
    print(f"  - Total tasks: {detail['total_tasks']}")
    print(f"  - Beginning files: {detail['beginning_files']}")
    print(f"  - Ending files: {detail['ending_files']}")

print(f"\nPhase 3 completed successfully!")
print(f"Results saved to: {config.get('INTERIM_DATA_DIR')}/phase3/")

Saved generated_checkpoints to interim_data/phase3/generated_checkpoints.json
Saved phase3_summary to interim_data/phase3/phase3_summary.json

Phase 3 Summary:

Checkpoint 1: Implement New Server Actions for Context Bundles
  - Tranches: 2
  - Total tasks: 9
  - Beginning files: 2
  - Ending files: 4

Checkpoint 2: Refactor Legacy `resolveContextSelectionAction`
  - Tranches: 1
  - Total tasks: 1
  - Beginning files: 5
  - Ending files: 5

Checkpoint 3: Implement Frontend State Management for Active Context Bundle
  - Tranches: 1
  - Total tasks: 1
  - Beginning files: 5
  - Ending files: 6

Checkpoint 4: Integrate Context Bundle ID into Frontend Operations and State
  - Tranches: 2
  - Total tasks: 2
  - Beginning files: 6
  - Ending files: 8

Checkpoint 5: Implement Frontend Display of Context Bundle Snapshots on Cards
  - Tranches: 1
  - Total tasks: 1
  - Beginning files: 8
  - Ending files: 10

Checkpoint 6: Update Documentation to Reflect Server-Action Architecture
  - Tranches: 